論文<br>
https://arxiv.org/abs/2207.04808<br>
<br>
GitHub<br>
https://github.com/JarrentWu1031/CCPL<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/ccpl_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GitHubからソースコードを取得

In [ ]:
%cd /content

!git clone https://github.com/JarrentWu1031/CCPL.git

## ライブラリのインストール

In [ ]:
%cd /content/CCPL

!pip install --upgrade gdown
!pip install -r requirements.txt

!pip install moviepy==0.2.3.5 imageio==2.4.1
!pip install yt-dlp

## ライブラリをインポート

In [ ]:
import os

from yt_dlp import YoutubeDL
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

from PIL import Image
import IPython

# 学習済みモデルのダウンロード

In [ ]:
%cd /content/CCPL
!mkdir -p pretrained/art pretrained/photo models

# https://drive.google.com/drive/folders/1XxhpzFqCVvboIyXKLfb2ocJZabPYu3pi
art_decoder_ckpt = 'pretrained/art/decoder_iter_160000.pth.tar'
if not os.path.exists(art_decoder_ckpt):
  !gdown --id 1QSTHPqhsyReU4BKtCQIWJC5tL1nXBUOt \
          -O {art_decoder_ckpt}

art_sct_ckpt = 'pretrained/art/sct_iter_160000.pth.tar'
if not os.path.exists(art_sct_ckpt):
  !gdown --id 1ccA2Slqr8wptQtFjs7kHCkemKxT642Ud \
          -O {art_sct_ckpt}

photo_decoder_ckpt = 'pretrained/photo/decoder_iter_160000.pth.tar'
if not os.path.exists(photo_decoder_ckpt):
  !gdown --id 1GothVHT9sWCwrggQWEdTchVu3FlPx9Fx \
          -O {photo_decoder_ckpt}

photo_sct_ckpt = 'pretrained/photo/sct_iter_160000.pth.tar'
if not os.path.exists(photo_sct_ckpt):
  !gdown --id 1LHAbK4gUlmY2QWHKzf6gfUOSiGKxvETv \
          -O {photo_sct_ckpt}


vgg_ckpt = 'models/vgg_normalised.pth'
if not os.path.exists(vgg_ckpt):
  !gdown --id 1EpkBA2K2eYILDSyPTt0fztz59UjAIpZU \
          -O {vgg_ckpt}

# テスト画像取得

In [ ]:
%cd /content/CCPL

!mkdir test_img

!wget -c https://www.pakutaso.com/shared/img/thumb/TSJ86_date20150208082618_TP_V4.jpg \
      -O test_img/test_01.jpg

# テスト動画取得

In [ ]:
video_url = 'https://www.youtube.com/watch?v=8tOIjOPZ5sA' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  31#@param {type:"integer"}
end_sec =  33#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
%cd /content/CCPL

!mkdir -p test_video/frames

download_resolution = 720
full_video_path = '/content/CCPL/test_video/full_video.mp4'
input_clip_path = '/content/CCPL/test_video/clip_video.mp4'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

# Style Transfer Image

In [ ]:
%cd /content/CCPL

# testing_mode art or pho
!python ./test.py \
  --vgg models/vgg_normalised.pth \
  --SCT pretrained/photo/sct_iter_160000.pth.tar \
  --decoder pretrained/photo/decoder_iter_160000.pth.tar \
  --content test_img/test_01.jpg \
  --style input/style/sketch.png \
  --testing_mode pho \
  --output outputs/imgs

In [ ]:
im = Image.open('/content/CCPL/outputs/imgs/test_01_stylized_sketch.jpg')
IPython.display.display(im)

# Style Transfer Video

## Frame画像生成

In [ ]:
%cd /content/CCPL

!ffmpeg -i {input_clip_path} test_video/frames/%06d.png

In [ ]:
!sed -i /content/CCPL/test_video_frame.py -e "s/transforms.Scale/transforms.Resize/g"

In [ ]:
!python ./test_video_frame.py \
  --vgg models/vgg_normalised.pth \
  --SCT pretrained/art/sct_iter_160000.pth.tar \
  --decoder pretrained/art/decoder_iter_160000.pth.tar \
  --content_dir test_video/frames \
  --style_path input/style/sketch.png \
  --testing_mode art \
  --output outputs/videos/art

In [ ]:
clip = VideoFileClip("outputs/videos/art/transfer.avi")
clip = resize(clip, height=420)
clip.ipython_display()

In [ ]:
!python ./test_video_frame.py \
  --vgg models/vgg_normalised.pth \
  --SCT pretrained/photo/sct_iter_160000.pth.tar \
  --decoder pretrained/photo/decoder_iter_160000.pth.tar \
  --content_dir test_video/frames \
  --style_path input/style/sketch.png \
  --testing_mode pho \
  --preserve_color \
  --output outputs/videos/pho

In [ ]:
clip = VideoFileClip("outputs/videos/pho/transfer.avi")
clip = resize(clip, height=420)
clip.ipython_display()